<a href="https://colab.research.google.com/github/seawavve/CNN_wavve/blob/main/efficientWorkin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from matplotlib import pyplot
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import datasets
import numpy as np
import sys

print('Python version : ', sys.version)
print('Keras version : ', keras.__version__)

# def MBConv(x, squeeze=16, expand=64):
#   m = layers.Conv2D(squeeze, (1,1), activation='relu', padding = 'same')(x)
#   m = layers.Conv2D(squeeze, (3,3), activation='relu', padding = 'same')(m)
#   m = layers.Conv2D(expand, (1,1), activation='relu', padding = 'same')(m)
#   return layers.Add()([m, x])

def MBConv(x, expand=64, squeeze=16):
  m = layers.Conv2D(expand, (1,1), activation='relu')(x)
  m = layers.DepthwiseConv2D((3,3), activation='relu')(m)
  m = layers.Conv2D(squeeze, (1,1), activation='relu')(m)
  return layers.Add()([m, x])

img_rows = 32   
img_cols = 32
(x_train, y_train), (x_test, y_test) =keras.datasets.cifar10.load_data()  

input_shape = (img_rows, img_cols,3)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)  
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
epochs = 3
filename='checkpoint.h5'.format(epochs)
early_stopping=EarlyStopping(monitor='val_loss',mode='min',patience=15,verbose=1)          
checkpoint=ModelCheckpoint(filename,monitor='val_loss',verbose=1,save_best_only=True,mode='auto')   

inputs = keras.Input(shape = input_shape, name = 'input')
output = layers.Conv2D(filters = 32, kernel_size = [3,3], padding = 'same', activation='relu')(inputs)
output= MBConv(output, squeeze=16, expand=64)
output = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding = 'same')(output)
k= layers.Dropout(rate=0.2)(output)
output= layers.Conv2D(filters=32, kernel_size=[3,3], padding='same', activation='relu')(output)
output=layers.Add()([k,output])

output = layers.GlobalAveragePooling2D()(output)
output = layers.Dense(1000, activation ='relu', name='fc0')(output)
dropout = layers.Dropout(rate=0.2)(output)
output = layers.Dense(10, activation = 'softmax', name='output')(dropout)

model = keras.Model(inputs = inputs, outputs = output)
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=epochs, verbose=1, validation_data=(x_test, y_test),callbacks=[checkpoint,early_stopping]) #학습

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:',  score[0])
print('Test accuracy:', score[1])

Python version :  3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
Keras version :  2.4.0
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


ValueError: ignored